# Wissensaggregator Mittelalter und frühe Neuzeit
WIAGweb2

## Daten für Bischöfe aus dem Digitalen Personenregister einlesen

Finde Einträge, die von der Bischofsdatenbank referenziert werden (sofern die verweisenden Bischöfe online sind).

Übertrage Daten aus der GS-online. Die Daten werden aus der produktiven Datenbank auf eine lokale Datenbank eingelesen: `gso`

[Literatur](#Literatur)  
[Bischöfe](#Bischöfe)
- [Einträge in `item`](#Einträge-in-item)
- [Daten eintragen](#Daten-eintragen)
- [Orden eintragen](#Orden-eintragen)
- [Literaturverweise eintragen](#Literaturverweise-eintragen)
- [Externe IDs eintragen](#Externe-IDs-eintragen)
- [Namensvarianten eintragen](#Namensvarianten-eintragen)
- [Suchvarianten eintragen (`name_lookup`)](#Suchvarianten-eintragen-(name_lookup))

[Ämter](#Ämter)

Quelle:  

| Datum | Datenbank |
| --- | --- |
| 2022-02-25 | `gso_in_202202` |
| 2022-05-03 | `gso_in_202204` | 
| 2022-05-12 | `gso_in_202205` | 
| 2022-05-26 | `gso_in_202207` | 
| 2022-10-19 | `gso_in_202210` | 
| 2022-11-03 | `gso_in_202210` | 
| 2022-11-23 | `gso_in_202211` |

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
gs_db = "gso_in_202211"

"gso_in_202211"

In [2]:
wds_path="../.."

"../.."

In [3]:
cd(wds_path)

In [4]:
using Pkg

In [5]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [6]:
using Revise

In [7]:
using WiagDataSetup

In [8]:
using MySQL, DataFrames

In [9]:
Wds = WiagDataSetup

WiagDataSetup

In [10]:
Wds.setDBWIAG(user="georg", db="wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in")

In [11]:
item_type_bishop_gs = 9

9

*2022-03-22* einmalig  
Eintrag in `item_type`

In [12]:
table_name = "item_type";
sql = "SELECT * FROM $(table_name)
LIMIT 0";
df_type = Wds.sql_df(sql)

,id,comment,name,note,created_by,date_created,changed_by,date_changed
,Int32,String?,String?,String?,Int32,DateTim…,Int32,DateTim…?


In [11]:
using Dates

In [12]:
names(df_type)

LoadError: UndefVarError: df_type not defined

In [15]:
user_wiag_id = 1;
time_stamp = Dates.format(now(), dateformat"yyyy-mm-dd HH:MM");

In [17]:
user_id_gh = 7;

In [22]:
 rec = (id = item_type_bishop_gs, 
    comment = missing, 
    name = "Bischof GS", 
    note = "Bischof des Alten Reiches aus der Personendatenbank der Germania Sacra",
    id_public_key = missing,
    table_name = "person",
    name_app = "bishop",
    created_by = user_id_gh,
    date_created = Dates.now(),
    changed_by = user_id_gh,
    date_changed = Dates.now())

(id = 9, comment = missing, name = "Bischof GS", note = "Bischof des Alten Reiches aus der Personendatenbank der Germania Sacra", id_public_key = missing, table_name = "person", name_app = "bishop", created_by = 7, date_created = DateTime("2022-03-22T08:05:26.440"), changed_by = 7, date_changed = DateTime("2022-03-22T08:05:26.440"))

In [23]:
push!(df_type, rec)

,id,comment,name,note
,Int32,String?,String?,String?
1,9,missing,Bischof GS,Bischof des Alten Reiches aus der Personendatenbank der Germania Sacra


In [24]:
table_name = "item_type"
Wds.filltable!(table_name, df_type, clear_table = false)

┌ Info: Rows inserted: 1
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


1

Item Type ID

In [150]:
item_type_id = item_type_bishop_gs

9

## Löschungen für umfassendes Update

In [151]:
table_name = "id_external";
sql = "DELETE FROM $(table_name)
WHERE item_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM id_external\nWHERE item_id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 2363, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [152]:
table_name = "person_role_property";
sql = "DELETE FROM $(table_name)
WHERE person_role_id IN
(SELECT id FROM person_role
WHERE person_id in (SELECT id from item WHERE item_type_id = $(item_type_id)))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role_property\nWHERE person_role_id IN\n(SELECT id FROM person_role\nWHERE person_id in (SELECT id from item WHERE item_type_id = 9))", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [153]:
table_name = "person_role";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 2158, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [154]:
table_name = "person";
sql = "DELETE FROM $(table_name)
WHERE id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person\nWHERE id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 1361, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [155]:
table_name = "item_reference";
sql = "DELETE FROM $(table_name)
WHERE item_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM item_reference\nWHERE item_id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 374, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [156]:
table_name = "item";
sql = "DELETE FROM $(table_name)
WHERE item_type_id = $(item_type_id)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM item\nWHERE item_type_id = 9", 0, -1, 1361, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

## Literatur
Die Literatur ist dieselbe wie für die Domherren GS, wir brauchen sie aber nochmal mit einem neuen Wert für `item_type_id`.

In [19]:
table_name = gs_db * ".books"
sql = "SELECT id, titel AS title, autoren AS author, nummer AS number_vol, " *
"uri AS online_resource, kurztitel AS shorttitle , `order` as display_order " *
"FROM $(table_name)";
df_vol = Wds.sql_df(sql);

In [20]:
columns = [
    :id => :reference_id,
    :title => :full_citation,
    :author => :author_editor,
    :online_resource => :online_resource,
    :shorttitle => :title_short,
    :number_vol => :gs_volume_nr,
    :display_order => :display_order,
]

7-element Vector{Pair{Symbol, Symbol}}:
              :id => :reference_id
           :title => :full_citation
          :author => :author_editor
 :online_resource => :online_resource
      :shorttitle => :title_short
      :number_vol => :gs_volume_nr
   :display_order => :display_order

In [21]:
df_vol_db = select(df_vol, columns);

*2022-11-03* Ausgabe für bk als CSV

In [15]:
using CSV

In [17]:
csv_books = "C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-11-03\\books.csv"
CSV.write(csv_books, df_vol)

"C:\\Users\\georg\\Documents\\projekte-doc\\WIAGweb2\\data\\bischoefe-2022-11-03\\books.csv"

In [18]:
names(df_vol)

6-element Vector{String}:
 "id"
 "title"
 "author"
 "number_vol"
 "online_resource"
 "shorttitle"

In [24]:
df_vol_db[!, :item_type_id] .= item_type_bishop_gs;

Bestehende Einträge löschen

In [22]:
table_name = "reference_volume"
sql = "DELETE FROM $(table_name) WHERE item_type_id = $(item_type_bishop_gs)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM reference_volume WHERE item_type_id = 9", 0, -1, 89, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten eintragen

In [25]:
Wds.filltable!(table_name, df_vol_db, clear_table = false)

┌ Info: Rows inserted: 89
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


89

## Bischöfe

Betrachte nur Personen, deren Eintrag nicht gelöscht ist und die online sind. Die maßgebliche GSN steht in dem Datensatz mit dem kleinsten Wert der id.

Finde Einträge, die von der Bischofsdatenbank referenziert werden (sofern die verweisenden Bischöfe online sind!)

In [157]:
item_type_bishop = 4
db_ref = "wiag_in"
table_name = "item";
sql = "SELECT item.id as ep_item_id, e.value AS gsn
FROM $(db_ref).item 
JOIN $(db_ref).id_external AS e ON item.id = e.item_id
WHERE item_type_id = $(item_type_bishop)
AND authority_id = 200
AND item.is_online";
ep_gsn = Wds.sql_df(sql);

In [158]:
size(ep_gsn, 1)

2143

Lies aus `gsn` `item_id`

In [159]:
table_name = gs_db * ".gsn"
sql = "SELECT gsn.id as gsn_id, item_id, gsn.nummer, items.status 
FROM $(table_name)
JOIN $(gs_db).items ON item_id = items.id
AND NOT items.deleted AND items.status = 'online';"
item_all = Wds.sql_df(sql);

In [160]:
ep_item_gsn = innerjoin(ep_gsn, item_all, on = :gsn => :nummer);

In [161]:
size(ep_item_gsn, 1)

2143

In [162]:
names(ep_item_gsn)

5-element Vector{String}:
 "ep_item_id"
 "gsn"
 "gsn_id"
 "item_id"
 "status"

`persons`

In [163]:
table_name = gs_db * ".persons"
@info "" table_name
sql = "SELECT p.id as id_person_gso, item_id, " *
"namenspraefix, vorname, familienname, " *
"sterbedatum, geburtsdatum, `orden`, " *
"anmerkungen, titel, " * 
"gndnummer, viaf, " *
"familiennamenvarianten, vornamenvarianten, " *
"namenszusatz, " *
"zeitraum_von, zeitraum_bis " *
"FROM $(table_name) as p " *
"JOIN $gs_db.items ON items.id = p.item_id " *
"WHERE items.status = 'online' AND NOT items.deleted " *
"ORDER BY p.id";
df_p = Wds.sql_df(sql);

┌ Info: 
│   table_name = gso_in_202211.persons
└ @ Main In[163]:2


In [164]:
Wds.clean_up!(df_p)

In [165]:
one_valid(x, y) = !ismissing(x) | !ismissing(y)
subset!(df_p, [:vorname, :familienname] => ByRow(one_valid));

In [166]:
df_p_gsn = innerjoin(df_p, ep_item_gsn, on = :item_id);

In [167]:
size(df_p_gsn, 1)

2143

Die Bischöfe aus der Personendatenbank erhalten keine WIAG-ID

Vermeide Dopplungen. Es werden keine Bischöfe aufgenommen, die schon als Domherren referenziert sind

In [168]:
item_type_canon_gs = 6;
table_name = "item";
sql = "SELECT * FROM $(table_name) WHERE item_type_id = $(item_type_canon_gs)"
df_gs = Wds.sql_df(sql);

Wandle `id_person_gso` um in String für den Vergleich

In [169]:
transform!(df_p_gsn, :id_person_gso => ByRow(string) => :id_in_source);

In [170]:
df_p_gsn = antijoin(df_p_gsn, df_gs, on = :id_in_source);

In [171]:
size(df_p_gsn, 1)

1361

### Einträge in `item`

entsprechende Einträge in Tabelle `item` einfügen

`id` für 'Bischof GS' in `item_type`: 9

In [172]:
df_p_gsn[210:215, [:vorname, :familienname, :geburtsdatum, :sterbedatum, :id_in_source]]

Row,vorname,familienname,geburtsdatum,sterbedatum,id_in_source
,String?,String?,String?,String?,String
1,Drogo,missing,missing,967,207146
2,Everacrus,missing,missing,971,207224
3,Everger,missing,missing,999,207225
4,Engilmar,missing,missing,missing,207444
5,Volkold,missing,missing,missing,207772
6,Franco,missing,missing,903,207805


In [173]:
df_item = select(df_p_gsn, :id_in_source);

In [174]:
user_wiag_id = 40; # wiag

In [175]:
using Dates

Setze den Status auf "online", also dem entsprechenden Status in der Personendatenbank

In [176]:
insertcols!(
    df_item, 
    1,
    :item_type_id => item_type_bishop_gs,
    :edit_status => "online",
    :created_by => user_wiag_id,
    :date_created => Dates.now(),
    :changed_by => user_wiag_id,
    :date_changed => Dates.now(),
    :id_public => missing,
    :is_online => 1
);

In [177]:
df_item[17, :]

Row,item_type_id,edit_status,created_by,date_created,changed_by,date_changed,id_public,is_online,id_in_source
,Int64,String,Int64,DateTime,Int64,DateTime,Missing,Int64,String
17,9,importiert,40,2022-12-05T15:26:13.833,40,2022-12-05T15:26:13.833,missing,1,26346


In [178]:
names(df_item)

9-element Vector{String}:
 "item_type_id"
 "edit_status"
 "created_by"
 "date_created"
 "changed_by"
 "date_changed"
 "id_public"
 "is_online"
 "id_in_source"

Bestehende Einträge löschen  
Die Einträge in `item` sind bei einem Update als letztes zu löschen, da die anderen Tabellen von Tabelle `item` abhängen.

In [179]:
item_type_bishop_gs

9

In [180]:
table_name = "item";
sql = "DELETE FROM $(table_name) WHERE item_type_id = $(item_type_bishop_gs)"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM item WHERE item_type_id = 9", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [181]:
table_name = "item";
Wds.filltable!(table_name, df_item, clear_table = false)

┌ Info: Rows inserted: 1361
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


1361

### Daten eintragen

IDs auslesen

In [182]:
table_name = "item";
sql = "SELECT id, id_in_source, item_type_id FROM $(table_name) where item_type_id = ($item_type_id)"
df_idx = DBInterface.execute(Wds.dbwiag, sql) |> DataFrame;

In [183]:
size(df_idx)

(1361, 3)

In [184]:
df_ep_idx = innerjoin(df_idx, df_p_gsn, on = :id_in_source);

In [185]:
size(df_ep_idx, 1)

1361

Jahreszahlen ermitteln

In [186]:
parse_date_birth(s) = Wds.parsemaybe(s, :lower)

parse_date_birth (generic function with 1 method)

In [187]:
parse_date_death(s) = Wds.parsemaybe(s, :upper)

parse_date_death (generic function with 1 method)

In [188]:
transform!(df_ep_idx, :geburtsdatum => ByRow(parse_date_birth) => :num_date_birth);

In [189]:
transform!(df_ep_idx, :sterbedatum => ByRow(parse_date_death) => :num_date_death);

Orden zuordnen

In [190]:
sql = "SELECT id as religious_order_id, abbreviation FROM religious_order";
df_rel_order = Wds.sql_df(sql);

In [191]:
df_ep_idx = leftjoin(df_ep_idx, df_rel_order, on = :orden => :abbreviation, matchmissing = :notequal);

In [192]:
a_not_b(a, b) = !ismissing(a) && ismissing(b)

a_not_b (generic function with 1 method)

In [193]:
df_ep_no_match = subset(df_ep_idx, [:orden, :religious_order_id] => ByRow(a_not_b));

In [194]:
size(df_ep_no_match)

(1, 27)

In [195]:
df_ep_no_match[:, [:id, :id_in_source, :orden]]

Row,id,id_in_source,orden
,Int32,String?,String?
1,703007,84864,Wilhelmit


Der Orden ist nicht in der Liste und es scheint auch keine gängige Abkürzung zu geben.

In [196]:
columns = [
    :id => :id,
    :item_type_id => :item_type_id,
    :namenspraefix => :prefixname,
    :vorname => :givenname,
    :familienname => :familyname,
    :anmerkungen => :comment,
    :geburtsdatum => :date_birth,
    :sterbedatum => :date_death,
    :num_date_birth => :num_date_birth,
    :num_date_death => :num_date_death,
    :religious_order_id => :religious_order_id,
];

Einträge löschen (Korrektur/Update)

In [197]:
table_name = "person"
sql = "DELETE FROM $(table_name)
WHERE id IN (SELECT id FROM item WHERE item_type_id = $(item_type_bishop_gs))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person\nWHERE id IN (SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [198]:
Wds.filltable!("person", select(df_ep_idx, columns); clear_table=false)

┌ Info: Rows inserted: 1361
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


1361

### Orden eintragen
*2022-01-19* Die (vorläufige) Ordenstabelle basiert auf den Ordensangaben für Bischöfe und auf den Ordensangaben hier.

In [55]:
unique(df_p_gsn.orden)

17-element Vector{Union{Missing, String}}:
 "OESA"
 "OP"
 missing
 "OPraem"
 "OFM"
 "OCist"
 "OSB"
 "Wilhelmit"
 "OCarm"
 "OMel"
 "OCart"
 "OFMConv"
 "OFMCap"
 "OSA"
 "OT"
 "CanA"
 "SJ"

In [177]:
table_name = "religious_order"
sql = "SELECT abbreviation FROM $(table_name)"
df_orders = Wds.sql_df(sql);

Eintrag in `item`

*2022-02-25* Nur nötig, wenn die Orden neu angelegt werden.

In [141]:
table_name = "item";
sql = "SELECT * FROM $(table_name) LIMIT 0";
df_item = Wds.sql_df(sql);

In [135]:
id_next = 22;

In [136]:
item_type_order = 7;
user_wiag_id = 7; # Georg
time_stamp = Dates.format(now(), dateformat"yyyy-mm-dd HH:MM");

In [142]:
df_item = select(df_item, Not(:id));

In [156]:
table_name = "item"
sql = "INSERT INTO $(table_name) SET
    item_type_id = $(item_type_order),    
    id_in_source = $(id_next),
    is_deleted = 0,
    is_online = 0,
    created_by = $(user_wiag_id),
    date_created = '$(time_stamp)',
    changed_by = $(user_wiag_id),
    date_changed = '$(time_stamp)'"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "INSERT INTO item SET\n    item_type_id = 7,    \n    id_in_source = 22,\n    is_deleted = 0,\n    is_online = 0,\n    created_by = 7,\n    date_created = '2022-01-19 11:11',\n    changed_by = 7,\n    date_changed = '2022-01-19 11:11'", 0, -1, 1, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Lies `id_item` für religiöse Orden aus

In [233]:
table_name = "item"
sql = "SELECT id, id_in_source 
FROM $(table_name) 
WHERE id_in_source = $(id_next) and item_type_id = $(item_type_order)"
df_idx = Wds.sql_df(sql);

LoadError: UndefVarError: id_next not defined

In [160]:
item_id = df_idx[1, :id]

92259

In [161]:
table_name = "religious_order";
sql = "INSERT INTO $(table_name) SET id = $(item_id), abbreviation = 'CRVC'";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "INSERT INTO religious_order SET id = 92259, abbreviation = 'CRVC'", 0, -1, 1, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

### Literaturverweise eintragen

IDs wieder einlesen, falls hier neu gestartet wird.

In [199]:
table_name = "item";
sql = "SELECT id, id_in_source
FROM $(table_name)
WHERE item_type_id = $(item_type_id)";
df_idx = Wds.sql_df(sql);

In [200]:
size(df_idx, 1), size(df_p_gsn, 1)

(1361, 1361)

In [201]:
df_p_gsn[1:5, [:id_person_gso, :vorname]]

Row,id_person_gso,vorname
,Int32,String?
1,4918,Dietrich
2,5147,Gerhard
3,5222,Gottschalk
4,5324,Gumpert
5,5382,Heimo


In [202]:
transform!(df_p_gsn, :id_person_gso => ByRow(string) => :id_in_source);

In [203]:
df_ep_idx = innerjoin(df_p_gsn, df_idx, on = :id_in_source);

In [204]:
size(df_ep_idx, 1)

1361

In [205]:
table_name = gs_db * ".locations";
sql = "SELECT item_id as item_id_gs, seiten, book_id  FROM $(table_name) " * 
"WHERE item_status = 'online' AND NOT deleted " *
"GROUP BY item_id, book_id, seiten"
df_p_ref = Wds.sql_df(sql);

Verbinde die Literaturangaben mit den relevanten Bischöfen

In [206]:
columns = [:id => :item_id, :item_id => :item_id_gs]

2-element Vector{Pair{Symbol, Symbol}}:
      :id => :item_id
 :item_id => :item_id_gs

In [207]:
df_p_ref = innerjoin(df_p_ref, select(df_ep_idx, columns), on = :item_id_gs);

In [208]:
size(df_p_ref)

(3739, 4)

Übernimm für Bischöfe mit Kurzbiographie nur Literaturhinweise mit Kurzbiographien (`transfer_gso.sql`: 2021-04-26) Es ist freilich möglich, dass es mehrere Literaturverweise gibt, die eine Kurzbiographie enthalten.

In [209]:
is_bio(seiten) = contains(seiten, "<b>");

In [210]:
df_p_ref_bio = subset(df_p_ref, :seiten =>ByRow(is_bio));

In [211]:
size(df_p_ref_bio)

(374, 4)

Bilde die Differenzmenge in Bezug auf Bischöfe, nicht in Bezug auf Literaturverweise

In [73]:
is_item_bio(item_id) = item_id in unique(df_p_ref_bio.item_id)

is_item_bio (generic function with 1 method)

In [74]:
df_p_ref_mg_bio = subset(df_p_ref, :item_id => ByRow(!is_item_bio));

Einträge löschen für Korrekturen oder ein Update

In [212]:
item_type_bishop_gs

9

In [213]:
table_name = "item_reference";
sql = "DELETE FROM $(table_name) WHERE item_type_id = $(item_type_bishop_gs)";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM item_reference WHERE item_type_id = 9", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [214]:
insertcols!(df_p_ref_bio, 1, :item_type_id => item_type_bishop_gs);

In [215]:
insertcols!(df_p_ref_mg_bio, 1, :item_type_id => item_type_bishop_gs);

LoadError: ArgumentError: Column item_type_id is already present in the data frame which is not allowed when `makeunique=true`

In [216]:
columns = [
    :item_id => :item_id,
    :seiten => :page,
    :item_type_id => :item_type_id,
    :book_id => :reference_id,
]

4-element Vector{Pair{Symbol, Symbol}}:
      :item_id => :item_id
       :seiten => :page
 :item_type_id => :item_type_id
      :book_id => :reference_id

In [217]:
table_name = "item_reference"
Wds.filltable!(table_name, select(df_p_ref_bio, columns))

┌ Info: Rows inserted: 374
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


374

In [218]:
df_p_ref_bio[19:22, :]

Row,item_type_id,item_id_gs,seiten,book_id,item_id
,Int64,Int32,String,Int32,Int32
1,9,147376,<b>267f.</b>,37,703075
2,9,176247,"127, 385, <b>395f.</b>, 416, 435",44,703099
3,9,179540,"872, <b>1013</b>",46,703094
4,9,180189,"464, <b>1018–1019</b>",46,703092


*2022-02-25* Es werden nur noch Literaturangaben mit Kurzbiographie berücksichtigt.

In [219]:
len_page = length.(df_p_ref)

Row,item_id_gs,seiten,book_id,item_id
,Int64,Int64,Int64,Int64
1,1,2,1,1
2,1,3,1,1
3,1,9,1,1
4,1,13,1,1
5,1,9,1,1
6,1,3,1,1
7,1,7,1,1
8,1,6,1,1
9,1,2,1,1


In [220]:
table_name = "item_reference"
Wds.filltable!(table_name, select(df_p_ref_bio, columns))

┌ Info: Rows inserted: 374
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


374

### Externe IDs eintragen

In [86]:
using MySQL, DataFrames

In [114]:
Wds.setDBWIAG(user = "georg", db = "wiag_in")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [221]:
item_type_bishop_gs

9

Einträge löschen (für den Fall eines Fehlers oder für ein Update)

In [222]:
sql = "DELETE FROM id_external " *
"WHERE item_id IN (SELECT id FROM item WHERE item_type_id = $(item_type_bishop_gs))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM id_external WHERE item_id IN (SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [223]:
sql = "SELECT id, url_name_formatter FROM authority " *
"WHERE id IN (1, 2, 3, 4, 5, 200)"
Wds.sql_df(sql)

Row,id,url_name_formatter
,Int32,String?
1,1,Gemeinsame Normdatei (GND) ID
2,2,Wikidata
3,3,Wikipedia-Artikel
4,4,VIAF-ID
5,5,WIAG-ID
6,200,Personendatenbank der Germania Sacra


In [84]:
df_ep_idx[300:307, [:id, :id_in_source, :vorname, :familienname, :gsn_id, :gsn]]

Row,id,id_in_source,vorname,familienname,gsn_id,gsn
,Int32,String,String?,String?,Int32,String?
1,692522,218883,Johannes,Schöner,188722,052-02166-001
2,692523,232677,Salomo II.,missing,135142,056-01453-001
3,692524,232653,Rudhart,missing,135118,056-01429-001
4,692525,232652,Ruodelo,missing,135117,056-01428-001
5,692526,232582,Remedius,missing,135047,056-01358-001
6,692527,232569,Reginhard,missing,135034,056-01345-001
7,692528,232537,Poppo,missing,135002,056-01313-001
8,692529,232527,Pictavus,missing,134992,056-01303-001


**VIAF**, dann GND, GSN

In [224]:
authority_id = 4 # siehe oben

4

In [225]:
col = :viaf;
df_p_auth = dropmissing(df_ep_idx, col);
insertcols!(df_p_auth, 1, :authority_id => authority_id);
transform!(df_p_auth, col => :value);

In [226]:
size(df_p_auth)

(2, 25)

In [227]:
df_p_auth

Row,authority_id,id_person_gso,item_id,namenspraefix,vorname,familienname,sterbedatum,geburtsdatum,orden,anmerkungen,titel,gndnummer,viaf,familiennamenvarianten,vornamenvarianten,namenszusatz,zeitraum_von,zeitraum_bis,ep_item_id,gsn,gsn_id,status,id_in_source,id,value
,Int64,Int32,Int32,String?,String?,String?,String?,String?,String?,String?,String?,String?,String,String?,String?,String?,Int32?,Int32?,Int32,String?,Int32,String,String,Int32,String
1,4,324288,330086,von,Joseph Franz Anton,Auersperg,1795,1734,missing,missing,missing,116212772,30281842,missing,missing,missing,1734,1795,562081,082-00027-001,370025,online,324288,704171,30281842
2,4,325009,330999,missing,Johann Andreas,Seelmann,1789,1732,missing,missing,missing,104321628,444474,missing,missing,missing,1732,1789,562563,082-00940-001,370938,online,325009,704207,444474


In [228]:
columns = [
    :id => :item_id,
    :authority_id => :authority_id,
    :value => :value,
]

3-element Vector{Pair{Symbol, Symbol}}:
           :id => :item_id
 :authority_id => :authority_id
        :value => :value

In [229]:
table_name = "id_external"
Wds.filltable!("id_external", select(df_p_auth, columns))

┌ Info: Rows inserted: 2
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


2

**GND**

In [230]:
authority_id = 1 # siehe oben

1

In [231]:
col = :gndnummer
df_p_auth = dropmissing(df_ep_idx, col);
insertcols!(df_p_auth, 1, :authority_id => authority_id);
transform!(df_p_auth, col => :value);

In [232]:
size(df_p_auth)

(1000, 25)

In [233]:
table_name = "id_external"
Wds.filltable!("id_external", select(df_p_auth, columns))

┌ Info: Rows inserted: 1000
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


1000

**GSN**

In [234]:
authority_id = 200 # siehe oben

200

In [235]:
col = :gsn
df_p_auth = dropmissing(df_ep_idx, col);
insertcols!(df_p_auth, 1, :authority_id => authority_id);
transform!(df_p_auth, col => :value);

In [236]:
table_name = "id_external"
Wds.filltable!("id_external", select(df_p_auth, columns))

┌ Info: Rows inserted: 1361
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


1361

### Namensvarianten eintragen

In [237]:
using MySQL, DataFrames

In [238]:
col = :familiennamenvarianten;
df_fnv = dropmissing(df_ep_idx, col);

In [239]:
size(df_fnv)

(221, 23)

In [240]:
df_exp = Wds.expand_column(df_fnv, col, delim = ",");

In [241]:
size(df_exp)

(283, 23)

In [242]:
df_exp[150:157, [:vorname, :familienname, col]]

Row,vorname,familienname,familiennamenvarianten
,String?,String?,String
1,Lothar Friedrich,Nalbach,Nelbach
2,Johann Heinrich,Anethan,Anethanus
3,Georg,Helffenstein,Helfenstein
4,Boemund I.,Warsberg,Saarbrücken-Warsberg
5,Arnold I.,Valcourt,Walcourt
6,Arnold I.,Valcourt,Vallecourt
7,Albero,Montreuil,Münsterol
8,Albero,Montreuil,Munsterol


Spalte `lang` hat den Default-Wert 'de'.

In [243]:
columns = [
    :id => :person_id,
    :familiennamenvarianten => :name,
]

2-element Vector{Pair{Symbol, Symbol}}:
                     :id => :person_id
 :familiennamenvarianten => :name

Einträge löschen (Korrekturen, Daten-Update)

In [244]:
item_type_bishop_gs

9

In [245]:
table_name = "familyname_variant";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_bishop_gs))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM familyname_variant\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [246]:
table_name = "familyname_variant"
Wds.filltable!(table_name, select(df_exp, columns))

┌ Info: Rows inserted: 283
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


283

Vornamen

In [247]:
col = :vornamenvarianten;
df_gnv = dropmissing(df_ep_idx, col);

In [248]:
size(df_gnv)

(430, 23)

In [249]:
df_exp = Wds.expand_column(df_gnv, col, delim = r", *");

In [250]:
size(df_exp)

(572, 23)

In [251]:
df_exp[150:157, [:vorname, :familienname, col]]

Row,vorname,familienname,vornamenvarianten
,String?,String?,String
1,Sylvester,Pflieger,Silvester
2,Gottschalk,missing,Gotescalc
3,Gottschalk,missing,Goteschalh
4,Gerhoh,missing,Gerho
5,Gerhoh,missing,Kerhoh
6,Gerhoh,missing,Gerhohus
7,Gerhoh,missing,Gerhoch
8,Gerhoh,missing,Geroch


In [252]:
columns = [
    :id => :person_id,
    col => :name,
]

2-element Vector{Pair{Symbol, Symbol}}:
                :id => :person_id
 :vornamenvarianten => :name

Einträge löschen (Korrekturen, Daten-Update)

In [253]:
item_type_bishop_gs

9

In [254]:
table_name = "givenname_variant";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_bishop_gs))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM givenname_variant\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [116]:
table_name = "givenname_variant"
Wds.filltable!(table_name, select(df_exp, columns))

┌ Info: Rows inserted: 572
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


572

### Suchvarianten eintragen (name_lookup)

Nach den Bischöfen aus der Personendatenbank wird nicht gesucht.

## Ämter

In [255]:
table_name = gs_db * ".offices";
sql = "SELECT * FROM $(table_name)
WHERE deleted = 0";
df_pr = Wds.sql_df(sql);

In [256]:
Wds.clean_up!(df_pr);

In [257]:
size(df_pr)

(73463, 22)

### Bischöfe zuordnen

In [258]:
transform!(df_pr, :person_id => ByRow(string) => :id_in_source);

In [259]:
table_name = "item";
sql = "SELECT id AS item_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_bishop_gs)";
df_idx = Wds.sql_df(sql);

In [260]:
df_prp = innerjoin(df_pr, df_idx, on = :id_in_source);

In [261]:
size(df_prp), size(unique(df_prp.item_id))

((2158, 24), (1356,))

In [262]:
person_id_mg_oc = setdiff(df_idx.item_id, df_prp.item_id)

5-element Vector{Int32}:
 702933
 703025
 703329
 703549
 704167

In [263]:
df_mg_bezeichnung = subset(df_prp, :bezeichnung => ByRow(ismissing));

In [264]:
df_mg_bezeichnung[:, [:id, :person_id, :item_id, :art, :ort, :dioezese, :bezeichnung]]

0×7 DataFrame

*2022-07-26* keine Elemente

In [265]:
subset(df_prp, :person_id => ByRow(isequal(317915)))[:, [:id, :person_id, :item_id, :art, :ort, :dioezese, :bezeichnung]]

0×7 DataFrame

In [266]:
dropmissing!(df_prp, :bezeichnung);

In [267]:
size(df_prp)

(2158, 24)

### Rollen/Ämter zuordnen

In [268]:
table_name = "role"
sql = "SELECT name as role_name, role.id as role_id
FROM $(table_name)"
df_role = Wds.sql_df(sql);

In [269]:
df_prpr = leftjoin(df_prp, df_role, on = :bezeichnung => :role_name);

In [270]:
df_mg_role_id = subset(df_prpr, :role_id => ByRow(ismissing));

In [271]:
size(df_prpr, 1), size(df_mg_role_id, 1)

(2158, 161)

Es gibt noch kein vollständiges Mapping von Ämtern auf normierte Ämter. Die Suche der Webanwendung berücksichtigt aber auch die ursprünglichen Bezeichnungen (person_role.role_name).

### Bistümer zuordnen

In [272]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

In [273]:
df_prprd = leftjoin(df_prpr, df_dioc, on = :dioezese => :diocese_name, matchmissing = :notequal);

In [274]:
size(df_prprd)

(2158, 26)

### Institutionen zuordnen

`klosterid` entspricht `institution.id_gsn`

In [275]:
sql = "SELECT id AS institution_id, name as institution_name, id_gsn FROM institution"
df_inst = Wds.sql_df(sql);

In [276]:
int_or_missing(x) = ismissing(x) ? missing : parse(Int, x);
transform!(df_prprd, :klosterid => ByRow(int_or_missing) => :klosterid);

In [277]:
df_prpm = leftjoin(df_prprd, df_inst, on = :klosterid => :id_gsn, matchmissing = :notequal);

In [278]:
size(df_prpm)

(2158, 28)

In [279]:
count(!ismissing, df_prpm.institution_id)

279

### Numerische Daten eintragen

In [280]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
transform!(df_prpm, :von => ByRow(num_date_begin) => :num_date_begin);

In [281]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper);
transform!(df_prpm, :bis => ByRow(num_date_end) => :num_date_end);

In [282]:
transform!(df_prpm, :bis => ByRow(Wds.parse_year_sort) => :date_sort_key);

In [283]:
function update_col(from, to)
    cand = Wds.parse_year_sort(from)
    if ismissing(to) || cand < to
        return cand
    else
        return to
    end
end

update_col (generic function with 1 method)

In [284]:
transform!(df_prpm, [:von, :date_sort_key] => ByRow(update_col) => :date_sort_key);

In [285]:
item_type_bishop_gs

9

In [286]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_bishop_gs))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag_in"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 9)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [287]:
columns = [
    :dioezese => :diocese_name,
    :von => :date_begin,
    :bis => :date_end,
    :anmerkung => :note,
    :bezeichnung => :role_name,
    :item_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :institution_name => :institution_name,    
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key
];

In [288]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prpm, columns))

┌ Info: Rows inserted: 2158
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1216


2158